# Lannet Internship Assignment - 2020

> User can input any dataset and we have to detect what’s date, what’s character etc. Since no data is perfect, a very simple data cleaning code won’t be able to read most of the variables and will remove those variables by the time it comes to predictive modelling. In case of any questions, You can take basic assumptions if you want to, we want to see how innovative you think. We want to see your thinking abilities and how much big you can think from this given information.

> ### **Things to consider**
    • Date column might have some invalid entries in them 
    • Date can be of different format throughout the column
    • Code should be efficient and fast
    • Make a dummy dataset by yourself to test out your logic
    • Code should be well commented and easy to interpret
    • Use google Colab
    • Code should be robust enough to run on any dataset
    • To test out the logic we will pass a random dataframe into your function

## Assumptions
- All these functions operate on pandas objects only,like pandas dataframe.
- The main date formats that are to be considered - dd/mm/yyyy,dd/mm/yy,yyyy/mm/dd,dd month yyyy, month dd yyyy.
- NO time objects will be entertained anywhere.
- StackOverflow will be used extensively.
- Timezones will be ignored completely.

> Libraries being used :
- `import numpy as np`
- `import pandas as pd`
- `from itertools import combinations`

In [68]:
import pandas as pd
import numpy as np
from itertools import combinations

> Dummy Data Preparation 

In [3]:
dat1 = pd.read_csv('~/Downloads/energydata_complete.csv',parse_dates=[0])

dat2=pd.read_csv('~/Downloads/amzn-anon-access-samples/amzn-anon-access-samples-history-2.0.csv',parse_dates=['REQUEST_DATE','AUTHORIZATION_DATE'])

In [4]:
#dat21 = dat2.iloc[:,3:5]
dat1['date'] = dat1['date'].dt.date
dat2['REQUEST_DATE'] = dat2['REQUEST_DATE'].dt.date
dat2['AUTHORIZATION_DATE']=pd.to_datetime(dat2['AUTHORIZATION_DATE'],errors='coerce').dt.date

In [44]:
data = pd.concat([dat1['date'],dat2.iloc[:,3:5].head(dat1.shape[0] - 1)], axis = 1)

data = data.astype('datetime64').dropna()

In [75]:
data.to_csv('dummy_date_data')

### 1.1 Write a function in python that identify which columns have date in them.
> Assumption - Data has been cleaned and the date columns has been parsed correctly

In [48]:
def date_cols(df):
    return list(df.select_dtypes('datetime').columns)

### 1.2 Using these date columns make new columns which are difference between these columns taking 2 at a time. (difference of days between dates).
> For instance - Data set contains 4 date columns which are start date, end date, DOB and Date of promotion. Then you will form 6 new columns containing difference of these date by taking 2 date at a time. One of the new column would be DOB - end date. But the data can contain any number of date columns (dataset has n number of date columns).
-------------------------------------------------------------------------------

In [67]:
def col_diff(df):
    dummy_df = pd.DataFrame()
    date_columns = date_cols(df)
    date_col_tups=list(combinations(date_columns, 2))
    for date_col_tup in date_col_tups:
        dummy_df[date_col_tup[0]+'-'+date_col_tup[1]] = (df[date_col_tup[0]] - df[date_col_tup[1]]).abs()  / np.timedelta64(1, 'D')
    return dummy_df

### 1.3 Drop all the original columns containing the date and just keep the newly computed columns
-----

In [66]:
def new_computed_df(df):
    date_columns = date_cols(df)
    dummy_df = col_diff(df)
    dummy_df_2 = df.drop(date_columns,axis=1)
    new_df = pd.concat([dummy_df_2,dummy_df],axis=1)
    return new_df

# Testing

In [69]:
data.head()

,date,REQUEST_DATE,AUTHORIZATION_DATE
0,2016-01-11,2005-03-25,2005-03-25
1,2016-01-11,2005-03-25,2005-03-25
2,2016-01-11,2005-04-04,2005-04-06
3,2016-01-11,2005-04-04,2005-04-06
4,2016-01-11,2005-04-04,2005-04-06


In [70]:
date_cols(data)

['date', 'REQUEST_DATE', 'AUTHORIZATION_DATE']

In [72]:
col_diff(data).head()

,date-REQUEST_DATE,date-AUTHORIZATION_DATE,REQUEST_DATE-AUTHORIZATION_DATE
0,3944.0,3944.0,0.0
1,3944.0,3944.0,0.0
2,3934.0,3932.0,2.0
3,3934.0,3932.0,2.0
4,3934.0,3932.0,2.0


In [74]:
new_computed_df(data).head()

,date-REQUEST_DATE,date-AUTHORIZATION_DATE,REQUEST_DATE-AUTHORIZATION_DATE
0,3944.0,3944.0,0.0
1,3944.0,3944.0,0.0
2,3934.0,3932.0,2.0
3,3934.0,3932.0,2.0
4,3934.0,3932.0,2.0
